In [1]:
import pandas as pd
import numpy as np
import json

In [321]:
!ls ./data

lda_fit.ipynb train.json


In [2]:
with open('./data/train.json', 'r') as f:
    data = json.load(f)

In [3]:
data = pd.DataFrame(data)
data.head()

,id,sentiment,text
0,1945,negative,Досудебное расследование по факту покупки ЕНПФ...
1,1957,negative,Медики рассказали о состоянии пострадавшего му...
2,1969,negative,"Прошел почти год, как железнодорожным оператор..."
3,1973,negative,По итогам 12 месяцев 2016 года на территории р...
4,1975,negative,Астана. 21 ноября. Kazakhstan Today - Агентств...


In [4]:
data['sentiment'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [5]:
y = data['sentiment'].to_numpy()
y[y == 'negative'] = -1
y[y == 'positive'] = 1
y[y == 'neutral'] = 0

In [6]:
from pymystem3 import Mystem
from nltk.corpus import stopwords
import re
from nltk import word_tokenize

In [7]:
stopw = stopwords.words('russian')

In [8]:
X = data['text'].to_numpy()

In [9]:
np.random.choice(X)

'\nЗа два с половиной года ЕНПФ увеличил инвестиции в развитие реального сектора экономики в 2 раза. За этот же период чистый инвестиционный доход к общему объему накоплений вырос с 20 до 36%, сообщает Finprom.kz\n\n\n\nПенсионная система активно инвестирует в реальный сектор экономики*. С момента консолидации пенсионных активов в ЕНПФ сумма инвестиций в инструменты эмитентов реального сектора экономики (включая институты развития) выросла больше чем в 2 раза.\n\nВ настоящий момент стоимость таких инвестиций в портфеле ЕНПФ превысила 1 триллион тенге.\n\n\n\nС начала 2014 года до августа текущего года сумма пенсионных активов ЕНПФ выросла на 63%, в том числе объем инвестированных активов - на 73%.\n\nЧистый инвестиционный доход фонда за этот период составил 1,3 триллиона тенге.\n\n\n\nСредства ЕНПФ в настоящий момент - мощный источник финансирования форсированного развития экономики страны в кризисный период.\n\nНаибольший объем пенсактивов в реальном секторе вложен в инструменты отрас

In [10]:
stemmer = Mystem()

In [11]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


def clean_string(string):    
    punc_free = re.sub('[^\w| ]', ' ', string)
    digit_free = re.sub(r'\d+', '', punc_free)
    cyrillic = re.sub(r"[^а-яА-ЯёЁ]+", " ", digit_free).lower()
    return cyrillic

In [12]:
class StemTfidfVectorizer(TfidfVectorizer):
    def __init__(self, stemmer, preprocessor, stop_words, ngram_range):
        super(StemTfidfVectorizer, self).__init__(
            preprocessor=preprocessor, stop_words=stop_words, ngram_range=ngram_range
        )
        self.stemmer = stemmer
        
    def build_analyzer(self):
        analyzer = super(StemTfidfVectorizer, self).build_analyzer()
        return lambda doc: (self.stemmer.lemmatize(word)[0] for word in analyzer(doc))

In [13]:
X.shape, y.shape

((8263,), (8263,))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [15]:
tfidf = StemTfidfVectorizer(stemmer=stemmer, preprocessor=clean_string, stop_words=stopw,
                           ngram_range=(1, 2))

In [16]:
tfidf.fit(X)

StemTfidfVectorizer(ngram_range=(1, 2),
                    preprocessor=<function clean_string at 0x13b2338c8>,
                    stemmer=<pymystem3.mystem.Mystem object at 0x130292f60>,
                    stop_words=['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я',
                                'с', 'со', 'как', 'а', 'то', 'все', 'она',
                                'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же',
                                'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', ...])

In [17]:
X.shape, y.shape

((8263,), (8263,))

In [18]:
!ls

__pycache__   data          example.ipynb main.ipynb    nb.py


In [19]:
from nb import NBFeaturer 

In [20]:
lab = LabelEncoder()
lab.fit(y_train)

train_y = []
for i in range(3):
    train_y.append((lab.transform(y_train) == i).astype(int))

train_X = tfidf.transform(X_train)
test_X = tfidf.transform(X_test)
test_y = lab.transform(y_test)

In [22]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

nbf = NBFeaturer(alpha=1)
model = LinearSVC(C=2, max_iter=100)

p = pipeline = Pipeline([
    ('nbf', nbf),
    ('lr', model)
])

In [23]:
train_y

[array([0, 0, 0, ..., 0, 0, 0]),
 array([1, 1, 1, ..., 0, 1, 0]),
 array([0, 0, 0, ..., 1, 0, 1])]

In [26]:
pred = []
for i in range(3):
    p.fit(train_X, train_y[i])
    pred.append(p.decision_function(test_X))

In [27]:
accuracy_score(test_y, np.argmax(np.array(pred), axis=0))

0.7291414752116082